# Iberdrola Project - Phase 2 [Occupancy Detection]

# Test GPU (Optional)
Before Starting, kindly check the available GPU from the Google Server, GPU model and other related information. It might help!

In [1]:
import torch
print("Is CUDA enabled GPU Available?", torch.cuda.is_available())
print("GPU Number:", torch.cuda.device_count())
print("Current GPU Index:", torch.cuda.current_device())
print("GPU Type:", torch.cuda.get_device_name(device=None))
print("GPU Capability:", torch.cuda.get_device_capability(device=None))
print("Is GPU Initialized yet?", torch.cuda.is_initialized())
print(torch.__version__)

Is CUDA enabled GPU Available? True
GPU Number: 1
Current GPU Index: 0
GPU Type: NVIDIA GeForce RTX 3080 Ti Laptop GPU
GPU Capability: (8, 6)
Is GPU Initialized yet? True
2.1.2+cu121


In [2]:
import tensorflow as tf
print(tf.__version__)
print(tf.config.experimental.list_physical_devices())
print(tf.config.list_physical_devices('GPU'))

2024-08-29 20:57:47.795647: I tensorflow/core/util/port.cc:113] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-08-29 20:57:47.838931: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-08-29 20:57:47.838967: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-08-29 20:57:47.840738: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-08-29 20:57:47.857446: I tensorflow/core/platform/cpu_feature_guar

2.15.0
[PhysicalDevice(name='/physical_device:CPU:0', device_type='CPU'), PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]
[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]


2024-08-29 20:57:49.533028: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:887] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2024-08-29 20:57:49.544549: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:887] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2024-08-29 20:57:49.544576: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:887] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.


# Import Libraries

In [3]:
import os
import h5py
import scipy
import random
import pickle
import json
import configparser
import numpy as np
import pandas as pd
import seaborn as sns
import scipy.io as sio
import tensorflow as tf
import matplotlib.pyplot as plt
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score, classification_report, confusion_matrix
from sklearn.model_selection import train_test_split
from scipy.io import loadmat, savemat
from pathlib import Path
from tqdm import tqdm
from scipy import stats
from random import shuffle

In [4]:
%matplotlib inline
sns.set_style('white')

# Import and Prepare Data

## Import and Extract Data

In [5]:
fl_Data = h5py.File(os.path.join('Raw_Data/Phase_2_Exp_1/02_Dishwasher_Data_Matched_Occupancy.mat'), 'r')
print(fl_Data.keys())
X_Data = np.expand_dims(np.array(fl_Data['data_appliance']), axis=2)
Y_Data = np.int_(np.expand_dims(np.array(fl_Data['data_occupancy']), axis=2))
X_Data_shape = X_Data.shape
Y_Data_shape = Y_Data.shape
print(X_Data_shape)
print(Y_Data_shape)
sample_num = X_Data_shape[0]
segment_length = X_Data_shape[1]
num_channels = X_Data_shape[2]
# Check for NaNs and InFs
data = pd.Series(X_Data.ravel())
print(data.isna().any())
print(data.isin([np.inf, -np.inf]).any())
data = pd.Series(Y_Data.ravel())
print(data.isna().any())
print(data.isin([np.inf, -np.inf]).any())
del fl_Data
sample_num = X_Data.shape[0]
segment_length = X_Data.shape[1]
num_channels = X_Data.shape[2]
# Check for NaNs and InFs
print(pd.Series(X_Data.ravel()).isna().any())
print(pd.Series(X_Data.ravel()).isin([np.inf, -np.inf]).any())
print(pd.Series(Y_Data.ravel()).isna().any())
print(pd.Series(Y_Data.ravel()).isin([np.inf, -np.inf]).any())

<KeysViewHDF5 ['data_appliance', 'data_occupancy', 'house_number_array']>
(38, 86400, 1)
(38, 86400, 1)
False
False
False
False
False
False
False
False


Preprocess Data

In [6]:
# Generate Load Event Pulses per channel
print('##### - Generating Load Event Pulses from Raw Device Data - #####')
Data_Pulses = np.zeros((sample_num,segment_length,X_Data.shape[2]))
counter = 0
thresh1 = 0
for i in range(0,sample_num):
    X_Data_Temp = X_Data[i,:,:]
    for ii in range(0,X_Data.shape[2]):
        X_Data_Temp_Ch = X_Data_Temp[:,ii]
        for iii in range(0,segment_length):
            X_Data_Temp_Ch_Point = X_Data_Temp_Ch[iii]
            if X_Data_Temp_Ch_Point > thresh1:
                X_Data_Temp_Ch_Point = 1
            Data_Pulses[counter,iii,ii] = X_Data_Temp_Ch_Point
    counter = counter + 1
Data_Pulses = np.int_(Data_Pulses[0:counter,:,:])
print(Data_Pulses.shape)
print(np.unique(Data_Pulses))
# Curate Data Pulses
print('##### - Curating Pulses - #####')
Data_Pulses_Curated = np.zeros((sample_num,segment_length,Data_Pulses.shape[2]))
Data_Ocp_Curated = np.zeros((sample_num,segment_length,Y_Data.shape[2]))
thresh2 = 1800
thresh3 = 1800
counter = 0
for i in range(0,sample_num):
    Data_Pulses_Temp = Data_Pulses[i,:,:]
    Data_Ocp_Temp = Y_Data[i,:,:]
    Data_Ocp_Temp_SUM = np.sum(Data_Ocp_Temp)
    if (Data_Ocp_Temp_SUM <= np.round(0.01*segment_length)) or (Data_Ocp_Temp_SUM >= np.round(0.99*segment_length)):
        continue
    Data_Pulses_Curated[counter,:,:] = Data_Pulses_Temp
    Data_Ocp_Curated[counter,:,:] = Data_Ocp_Temp
    counter = counter + 1
Data_Pulses_Curated = np.int_(Data_Pulses_Curated[0:counter,:,:])
Data_Ocp_Curated = np.int_(Data_Ocp_Curated[0:counter,:,:])
# Print
print(Data_Pulses_Curated.shape)
print(Data_Ocp_Curated.shape)
print(np.unique(Data_Pulses_Curated))
print(np.unique(Data_Ocp_Curated))
# Save
hfilew = h5py.File('Occupancy_Data_Curated.h5','w')
hfilew.create_dataset('X_Data', data=Data_Pulses_Curated)
hfilew.create_dataset('Y_Data', data=Data_Ocp_Curated)
hfilew.close()

##### - Generating Load Event Pulses from Raw Device Data - #####
(38, 86400, 1)
[0 1]
##### - Curating Pulses - #####
(31, 86400, 1)
(31, 86400, 1)
[0 1]
[0 1]


Shuffle Data

In [23]:
# Import
fl_Data = h5py.File(os.path.join('Occupancy_Data_Curated.h5'), 'r')
print(fl_Data.keys())
# Extract
X_Data = np.array(fl_Data['X_Data'])
print(X_Data.shape)
Y_Data = np.array(fl_Data['Y_Data'])
print(Y_Data.shape)
del fl_Data
sample_num = X_Data.shape[0]
segment_length = X_Data.shape[1]
num_channels = X_Data.shape[2]
# Check for NaNs and InFs
print(pd.Series(X_Data.ravel()).isna().any())
print(pd.Series(Y_Data.ravel()).isin([np.inf, -np.inf]).any())
print(pd.Series(X_Data.ravel()).isna().any())
print(pd.Series(Y_Data.ravel()).isin([np.inf, -np.inf]).any())
# Shuffle
ind_list = [i for i in range(sample_num)]
shuffle(ind_list)
X_Data_Shuffled = X_Data[ind_list,:,:]
Y_Data_Shuffled = Y_Data[ind_list,:,:]
print(X_Data_Shuffled.shape)
print(Y_Data_Shuffled.shape)

<KeysViewHDF5 ['X_Data', 'Y_Data']>
(31, 86400, 1)
(31, 86400, 1)
False
False
False
False
(31, 86400, 1)
(31, 86400, 1)


Create Folds and Transform Data for Classification

In [24]:
# Configurations
segment_length = 3600  # 60 Minutes = 3600 Data Points (1 Hz sampling rate)
offset_sec = 60  # 120 seconds offset for overlapping

In [25]:
def transform_labels(A_Data, B_Data, segment_length, offset_amount, C1=True, C2=True):
    A_Data_New = np.zeros((20000, A_Data.shape[2], segment_length))
    B_Data_New = np.zeros((20000, B_Data.shape[2]), dtype=int)
    # Reshape Array
    A_Data_Reshaped = np.reshape(A_Data, (A_Data.shape[0]*A_Data.shape[1], A_Data.shape[2]))
    B_Data_Reshaped = np.reshape(B_Data, (B_Data.shape[0]*B_Data.shape[1], B_Data.shape[2]))
    ## Slide through and generate labels
    counter = 0
    for i in range(0,B_Data_Reshaped.shape[0]):
        if (i*offset_amount+segment_length) >= A_Data_Reshaped.shape[0]:
            continue
        A_Data_Temp = np.transpose(A_Data_Reshaped[i*offset_amount:i*offset_amount+segment_length,:])
        # A_Data_Temp_Mean = np.mean(A_Data_Temp)
        A_Data_Temp_VAR = np.var(A_Data_Temp)
        B_Data_Temp = B_Data_Reshaped[i*offset_amount:i*offset_amount+segment_length,:]
        B_Data_Temp_Mean = np.mean(B_Data_Temp)
        # B_Data_Temp_VAR = np.var(B_Data_Temp)
        B_Data_Temp_SUM = np.sum(B_Data_Temp)
        if C1 == True:
            if A_Data_Temp_VAR <= 0:
                continue
        if C2 == True:
            if (B_Data_Temp_SUM < np.round(0.01*segment_length)) or (B_Data_Temp_SUM > np.round(0.99*segment_length)):
                continue
        if B_Data_Temp_Mean > 0.5:
            B_Label = 1
        else:
            B_Label = 0
        if (pd.Series(A_Data_Temp.ravel()).isna().any() == False) and (pd.Series(A_Data_Temp.ravel()).isin([np.inf, -np.inf]).any() == False):
            A_Data_New[counter,:,:] = A_Data_Temp
            B_Data_New[counter,:] = B_Label
            counter = counter + 1
    A_Data_New = A_Data_New[0:counter,:,:]
    B_Data_New = B_Data_New[0:counter,:]
    return A_Data_New, B_Data_New

In [ ]:
# Fold 1
sample_num = X_Data_Shuffled.shape[0]
X_Train_F1 = X_Data_Shuffled[0:round((sample_num/5)*4),:,:]
X_Test_F1 = X_Data_Shuffled[round((sample_num/5)*4):,:,:]
Y_Train_F1 = Y_Data_Shuffled[0:round((sample_num/5)*4),:,:]
Y_Test_F1 = Y_Data_Shuffled[round((sample_num/5)*4):,:,:]
X_Train_F1_C, Y_Train_F1_C = transform_labels(X_Train_F1, Y_Train_F1, segment_length, offset_sec, True, True)
X_Test_F1_C, Y_Test_F1_C = transform_labels(X_Test_F1, Y_Test_F1, segment_length, offset_sec, True, True)
print('##### - Fold 1 - #####')
print(X_Train_F1_C.shape)
print(Y_Train_F1_C.shape)
print(X_Test_F1_C.shape)
print(Y_Test_F1_C.shape)
labels, counts = np.unique(Y_Train_F1_C, return_counts=True)
print(labels, counts)
labels, counts = np.unique(Y_Test_F1_C, return_counts=True)
print(labels, counts)
del X_Train_F1, Y_Train_F1, X_Test_F1, Y_Test_F1
hfilew = h5py.File('Occupancy_Data_Classification_Fold1.h5','w')
hfilew.create_dataset('X_Train', data=X_Train_F1_C)
hfilew.create_dataset('Y_Train', data=Y_Train_F1_C)
hfilew.create_dataset('X_Test', data=X_Test_F1_C)
hfilew.create_dataset('Y_Test', data=Y_Test_F1_C)
hfilew.close()
# Fold 2
X_Train_F21 = X_Data_Shuffled[0:round((sample_num/5)*3),:,:]
X_Train_F22 = X_Data_Shuffled[round((sample_num/5)*4):,:,:]
X_Train_F2 = np.concatenate((X_Train_F21,X_Train_F22), axis=0)
X_Test_F2 = X_Data_Shuffled[round((sample_num/5)*3):round((sample_num/5)*4),:,:]
Y_Train_F21 = Y_Data_Shuffled[0:round((sample_num/5)*3),:,:]
Y_Train_F22 = Y_Data_Shuffled[round((sample_num/5)*4):,:,:]
Y_Train_F2 = np.concatenate((Y_Train_F21,Y_Train_F22), axis=0)
Y_Test_F2 = Y_Data_Shuffled[round((sample_num/5)*3):round((sample_num/5)*4),:,:]
X_Train_F2_C, Y_Train_F2_C = transform_labels(X_Train_F2, Y_Train_F2, segment_length, offset_sec, True, True)
X_Test_F2_C, Y_Test_F2_C = transform_labels(X_Test_F2, Y_Test_F2, segment_length, offset_sec, True, True)
print('##### - Fold 2 - #####')
print(X_Train_F2_C.shape)
print(Y_Train_F2_C.shape)
print(X_Test_F2_C.shape)
print(Y_Test_F2_C.shape)
labels, counts = np.unique(Y_Train_F2_C, return_counts=True)
print(labels, counts)
labels, counts = np.unique(Y_Test_F2_C, return_counts=True)
print(labels, counts)
del X_Train_F2, Y_Train_F2, X_Test_F2, Y_Test_F2
hfilew = h5py.File('Occupancy_Data_Classification_Fold2.h5','w')
hfilew.create_dataset('X_Train', data=X_Train_F2_C)
hfilew.create_dataset('Y_Train', data=Y_Train_F2_C)
hfilew.create_dataset('X_Test', data=X_Test_F2_C)
hfilew.create_dataset('Y_Test', data=Y_Test_F2_C)
hfilew.close()
# Fold 3
X_Train_F31 = X_Data_Shuffled[0:round((sample_num/5)*2),:,:]
X_Train_F32 = X_Data_Shuffled[round((sample_num/5)*3):,:,:]
X_Train_F3 = np.concatenate((X_Train_F31,X_Train_F32), axis=0)
X_Test_F3 = X_Data_Shuffled[round((sample_num/5)*2):round((sample_num/5)*3),:,:]
Y_Train_F31 = Y_Data_Shuffled[0:round((sample_num/5)*2),:,:]
Y_Train_F32 = Y_Data_Shuffled[round((sample_num/5)*3):,:,:]
Y_Train_F3 = np.concatenate((Y_Train_F31,Y_Train_F32), axis=0)
Y_Test_F3 = Y_Data_Shuffled[round((sample_num/5)*2):round((sample_num/5)*3),:,:]
X_Train_F3_C, Y_Train_F3_C = transform_labels(X_Train_F3, Y_Train_F3, segment_length, offset_sec, True, True)
X_Test_F3_C, Y_Test_F3_C = transform_labels(X_Test_F3, Y_Test_F3, segment_length, offset_sec, True, True)
print('##### - Fold 3 - #####')
print(X_Train_F3_C.shape)
print(Y_Train_F3_C.shape)
print(X_Test_F3_C.shape)
print(Y_Test_F3_C.shape)
labels, counts = np.unique(Y_Train_F3_C, return_counts=True)
print(labels, counts)
labels, counts = np.unique(Y_Test_F3_C, return_counts=True)
print(labels, counts)
del X_Train_F3, Y_Train_F3, X_Test_F3, Y_Test_F3
hfilew = h5py.File('Occupancy_Data_Classification_Fold3.h5','w')
hfilew.create_dataset('X_Train', data=X_Train_F3_C)
hfilew.create_dataset('Y_Train', data=Y_Train_F3_C)
hfilew.create_dataset('X_Test', data=X_Test_F3_C)
hfilew.create_dataset('Y_Test', data=Y_Test_F3_C)
hfilew.close()
# Fold 4
X_Train_F41 = X_Data_Shuffled[0:round((sample_num/5)*1),:,:]
X_Train_F42 = X_Data_Shuffled[round((sample_num/5)*2):,:,:]
X_Train_F4 = np.concatenate((X_Train_F41,X_Train_F42), axis=0)
X_Test_F4 = X_Data_Shuffled[round((sample_num/5)*1):round((sample_num/5)*2),:,:]
Y_Train_F41 = Y_Data_Shuffled[0:round((sample_num/5)*1),:,:]
Y_Train_F42 = Y_Data_Shuffled[round((sample_num/5)*2):,:,:]
Y_Train_F4 = np.concatenate((Y_Train_F41,Y_Train_F42), axis=0)
Y_Test_F4 = Y_Data_Shuffled[round((sample_num/5)*1):round((sample_num/5)*2),:,:]
X_Train_F4_C, Y_Train_F4_C = transform_labels(X_Train_F4, Y_Train_F4, segment_length, offset_sec, True, True)
X_Test_F4_C, Y_Test_F4_C = transform_labels(X_Test_F4, Y_Test_F4, segment_length, offset_sec, True, True)
print('##### - Fold 4 - #####')
print(X_Train_F4_C.shape)
print(Y_Train_F4_C.shape)
print(X_Test_F4_C.shape)
print(Y_Test_F4_C.shape)
labels, counts = np.unique(Y_Train_F4_C, return_counts=True)
print(labels, counts)
labels, counts = np.unique(Y_Test_F4_C, return_counts=True)
print(labels, counts)
del X_Train_F4, Y_Train_F4, X_Test_F4, Y_Test_F4
hfilew = h5py.File('Occupancy_Data_Classification_Fold4.h5','w')
hfilew.create_dataset('X_Train', data=X_Train_F4_C)
hfilew.create_dataset('Y_Train', data=Y_Train_F4_C)
hfilew.create_dataset('X_Test', data=X_Test_F4_C)
hfilew.create_dataset('Y_Test', data=Y_Test_F4_C)
hfilew.close()
# Fold 5
sample_num = X_Data_Shuffled.shape[0]
X_Train_F5 = X_Data_Shuffled[round((sample_num/5)*1):,:,:]
X_Test_F5 = X_Data_Shuffled[0:round((sample_num/5)*1),:,:]
Y_Train_F5 = Y_Data_Shuffled[round((sample_num/5)*1):,:,:]
Y_Test_F5 = Y_Data_Shuffled[0:round((sample_num/5)*1),:,:]
X_Train_F5_C, Y_Train_F5_C = transform_labels(X_Train_F5, Y_Train_F5, segment_length, offset_sec, True, True)
X_Test_F5_C, Y_Test_F5_C = transform_labels(X_Test_F5, Y_Test_F5, segment_length, offset_sec, True, True)
print('##### - Fold 5 - #####')
print(X_Train_F5_C.shape)
print(Y_Train_F5_C.shape)
print(X_Test_F5_C.shape)
print(Y_Test_F5_C.shape)
labels, counts = np.unique(Y_Train_F5_C, return_counts=True)
print(labels, counts)
labels, counts = np.unique(Y_Test_F5_C, return_counts=True)
print(labels, counts)
del X_Train_F5, Y_Train_F5, X_Test_F5, Y_Test_F5
hfilew = h5py.File('Occupancy_Data_Classification_Fold5.h5','w')
hfilew.create_dataset('X_Train', data=X_Train_F5_C)
hfilew.create_dataset('Y_Train', data=Y_Train_F5_C)
hfilew.create_dataset('X_Test', data=X_Test_F5_C)
hfilew.create_dataset('Y_Test', data=Y_Test_F5_C)
hfilew.close()

## Load Data and Save for Training

In [35]:
fold_num = 5

In [36]:
# Import
fl_Data = h5py.File(os.path.join(f'Occupancy_Data_Classification_Fold{fold_num}.h5'),'r')
print(fl_Data.keys())
X_Train = np.array(fl_Data['X_Train'])
Y_Train = np.array(fl_Data['Y_Train'])
X_Test = np.array(fl_Data['X_Test'])
Y_Test = np.array(fl_Data['Y_Test'])
train_data_dic = {"X_Train": X_Train,
                  "Y_Train": Y_Train,
                  "X_Test": X_Test,
                  "Y_Test": Y_Test,
                  "X_Val": X_Test,
                  "Y_Val": Y_Test,
                  }
savemat(f"Data/Data_Fold_{fold_num}.mat", train_data_dic, format='5')

<KeysViewHDF5 ['X_Test', 'X_Train', 'Y_Test', 'Y_Train']>


Garbage Collector

In [ ]:
import gc #Garbage Collector
fl_Data = None
X_Test = None
X_Train = None
X_Val = None
Y_Test = None
Y_Train = None
Y_Val = None
gc.collect()

0

# MAIN

## Configurations

In [37]:
# CREATE OBJECT
config_file = configparser.ConfigParser()
# ADD NEW SECTION AND SETTINGS
config_file["TRAIN"] = {
    'parentdir': '',  # Root or Parent Directory
    'datafile': 'Data',  # Folder containing the dataset
    'val_size': 0.0,  # Validation percentage for splitting
    'q_order': 3,  # q-order for the Self-ONN or Super-ONN Models
    'batch_size': 8,  # Batch Size, Change to fit hardware
    'lossType': 'MSE',  # loss function: 'SoftM_CELoss' or 'SoftM_MSE' or 'MSE'
    'optim_fc': 'SGD',  # 'Adam' or 'SGD'
    'lr': 0.0001,  # learning rate
    'stop_criteria': 'accuracy',  # Stopping criteria: 'loss' or 'accuracy'
    'n_epochs': 500,  # number of training epochs
    'epochs_patience': 10,
    'lr_factor': 0.1,  # lr_factor, if val loss did not decrease for a number of epochs (epochs_patience) then decrease learning rate by a factor of lr_factor
    'max_epochs_stop': 50,  # maximum number of epochs with no improvement in validation loss for early stopping
    'num_folds': 1,  # number of cross validation folds
    'load_model': False,  # load model: True or False
    'load_model_path': f'Results_Classification\ODNet_Occp_Classification\Fold_{fold_num}\ODNet_Occp_Classification_fold_{fold_num}.pt',  # specify path of pretrained model wieghts or set to False to train from scratch
    'model_to_load': 'ODNet',  # choose one of the following models: 'CNN_1' 'CNN_2' 'CNN_2' 'CNN_3' 'SelfResNet18' 'ResNet'
    'model_name': 'ODNet_Occp_Classification',  # choose a unique name for result folder
    'aux_logits': False,  # Required for models with auxilliary outputs (e.g., InceptionV3)  
    'fold_start': 1,  # The starting fold for training
    'fold_last': 5,  # The last fold for training
    'results_path': 'Results_Classification',  # main results folder
}

# SAVE CONFIG FILE
with open(r"Config_Train.ini", 'w') as configfileObj:
    config_file.write(configfileObj)
    configfileObj.flush()
    configfileObj.close()

print("Config file 'Config_Train.ini' created")

Config file 'Config_Train.ini' created


## Train

In [ ]:
%run -i Train.py

## Test

In [25]:
# CREATE OBJECT
config_file = configparser.ConfigParser()
# ADD NEW SECTION AND SETTINGS
config_file["TEST"] = {
    'parentdir': '',  # Root or Parent Directory
    'datafile': 'Data',  # Folder containing the dataset
    'batch_size': 8,  # Batch Size, Change to fit hardware
    'lossType': 'MSE',  # loss function: 'SoftM_CELoss' or 'SoftM_MSE' or 'MSE'
    'num_folds': 5,  # number of cross validation folds
    'CI': 0.9,  # Confidence interval (missied cases with probability>=CI will be reported in excel file)
    'load_model': False,  # specify path of pretrained model wieghts or set to False to train from scratch
    'load_model_path': f'Results_Classification\ODNet_Occp_Classification\Fold_1\ODNet_Occp_Classification_fold_1.pt',  # specify path of pretrained model wieghts or set to False to train from scratch
    'labeled_data': True,  # set to true if you have the labeled test set
    'model_name': 'ODNet_Occp_Classification',  # name of the saved model
    'aux_logits': False,  # Required for models with auxilliary outputs (e.g., InceptionV3)  
    'fold_start': 1,  # The starting fold for training
    'fold_last': 5,  # The last fold for training
    'N_steps': 1000,  # The last fold for training
    'results_path': 'Results_Classification',  # main results folder
}
# SAVE CONFIG FILE
with open(r"Config_Test.ini", 'w') as configfileObj:
    config_file.write(configfileObj)
    configfileObj.flush()
    configfileObj.close()
print("Config file 'Config_Test.ini' created")

Config file 'Config_Test.ini' created


In [ ]:
%run -i Test.py